In [41]:
import numpy as np 
import pandas as pd

from data import RandomData, AmazonBooks, ToyData, MovieLensData
from clustering import ClusteringModel

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data stuff 

In [42]:
ds = AmazonBooks(min_user_ratings=10).get_dataset(verbose=True)
train = ds['train']
val = ds['val']

loading raw data


ArrowIOError: Failed to open local file: data/amazonbooks/descriptions.feather , error: No such file or directory

In [3]:
print(train.keys())
train['user_product_ratings'].head()

dict_keys(['user_product_ratings', 'product_descriptions', 'product_reviews'])


,product_id,rating,user_id
0,6155,5.0,508
1,6155,5.0,557
2,2060,5.0,992
3,2060,5.0,606
4,6009,4.0,250


In [4]:
# len(train['user_product_ratings'].groupby('user_id')['rating'].count())

grouped_reviews = train['user_product_ratings'].groupby('user_id')['user_id', 'product_id', 'rating']
#     .aggregate(\
#         {'user_id':['mean'], 'review':lambda x: list(x)})
# grouped_reviews .columns = ['product_id', 'review']

In [91]:
# grouped_reviews.groups.keys()

In [202]:
# c_model = ClusteringModel()
# c_model._initialize_users(train)
# 


In [98]:
print(u.ratings)
g

[(2910, 4.0), (13886, 5.0), (8848, 5.0)]


,user_id,product_id,rating
943,1,2910,4.0
10947,1,13886,5.0
10951,1,8848,5.0


In [203]:
c.add_user(u)
c.new_center

{2910: (4.0, 1), 8848: (5.0, 1), 13886: (5.0, 1)}

In [204]:
c.recenter()
c.center

{2910: 4.0, 8848: 5.0, 13886: 5.0}

## centroid small tests  

In [35]:
from clustering import *

g = grouped_reviews.get_group(1)

c_model = ClusteringModel()
u = c_model.user(user_id= 0, grouped_data=g)
c = c_model.centroid(centroid_id = 0, initial_center={})

c.add_user(u)
c.recenter()

u2 =  c_model.user(user_id= 1, grouped_data=grouped_reviews.get_group(2))

print(c.center)
print(u.ratings)

c_model._distance(u, c)

ratings
{2910: 4.0, 13886: 5.0, 8848: 5.0}
ratings
{9511: 4.0, 9704: 3.0, 14354: 4.0, 9999: 4.0, 13520: 4.0, 3718: 5.0, 7601: 4.0, 4321: 5.0, 11129: 5.0}
{2910: 4.0, 13886: 5.0, 8848: 5.0}
{2910: 4.0, 13886: 5.0, 8848: 5.0}
just_centroid set()
just_user set()
0.0
0.0
0.0


0.0

In [39]:
c_model._distance(u2, c)

just_centroid {8848, 2910, 13886}
just_user set()
0.0
0.0
8.12403840463596


0.08124038404635961

In [37]:
# will clear and add users u2, u
c.add_user(u2)
c.add_user(u)
c.recenter()
print(c.center)

{9511: 4.0, 9704: 3.0, 14354: 4.0, 9999: 4.0, 13520: 4.0, 3718: 5.0, 7601: 4.0, 4321: 5.0, 11129: 5.0, 2910: 4.0, 13886: 5.0, 8848: 5.0}


In [16]:
for k in u.ratings:
    print(k)

2910
13886
8848


In [175]:
c_model.clusters[1].center

{96: 4.0,
 7116: 5.0,
 7361: 5.0,
 8285: 4.0,
 9516: 4.0,
 10141: 5.0,
 11536: 5.0,
 14914: 5.0}

## k_means perfing

In [70]:
from clustering import *

c_model = ClusteringModel(num_iterations=2)

In [63]:
def time_target():
    c_model = ClusteringModel(num_iterations=2)
    c_model.fit(train)

import timeit
timeit.Timer(stmt=time_target).timeit(number=30)


8.69278100000065

In [67]:
# for 10 
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


9.443330999999489

In [69]:
# for 50 
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


17.352155999999923

In [71]:
# for 70
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


20.161965999999666

In [72]:
# for 973
timeit.Timer(time_target, 'gc.enable()').timeit(number=30)


649.8086430000003

## converting results from clustering to be part of the database object
Goal take in map {user_id: cluster} and update the way we store data

in 

In [6]:
from clustering import *

c_model = ClusteringModel(num_clusters=90, num_iterations=70)
c_model.fit(train)


In [9]:
c_model.return_clusters()

{0: 12,
 1: 48,
 2: 19,
 3: 8,
 5: 8,
 6: 2,
 7: 8,
 8: 8,
 9: 5,
 10: 32,
 11: 64,
 12: 38,
 13: 8,
 14: 48,
 15: 8,
 16: 8,
 17: 28,
 19: 14,
 20: 84,
 21: 41,
 22: 8,
 23: 8,
 24: 18,
 25: 13,
 26: 34,
 27: 45,
 28: 84,
 29: 3,
 30: 3,
 31: 3,
 33: 68,
 34: 8,
 35: 8,
 37: 84,
 38: 8,
 39: 8,
 43: 2,
 44: 3,
 46: 3,
 48: 3,
 49: 53,
 50: 81,
 51: 61,
 52: 8,
 53: 12,
 54: 15,
 55: 8,
 56: 8,
 58: 8,
 59: 61,
 60: 40,
 63: 66,
 67: 87,
 68: 8,
 69: 79,
 70: 8,
 71: 8,
 72: 8,
 73: 53,
 74: 8,
 76: 45,
 77: 12,
 78: 38,
 79: 54,
 80: 65,
 81: 15,
 82: 87,
 83: 8,
 85: 3,
 86: 8,
 87: 73,
 88: 3,
 89: 61,
 90: 67,
 91: 33,
 92: 28,
 93: 56,
 94: 8,
 95: 71,
 97: 24,
 98: 54,
 99: 3,
 101: 8,
 102: 53,
 103: 8,
 104: 82,
 105: 8,
 106: 2,
 107: 8,
 108: 8,
 109: 8,
 110: 53,
 111: 3,
 115: 57,
 117: 8,
 118: 29,
 119: 2,
 120: 8,
 121: 27,
 122: 8,
 125: 8,
 126: 3,
 127: 8,
 128: 3,
 130: 3,
 132: 8,
 133: 8,
 136: 8,
 138: 72,
 139: 64,
 140: 0,
 141: 17,
 142: 7,
 143: 8,
 144: 8,
 1

In [10]:
cluster_centers = [c.center for c in c_model.clusters]

In [14]:
len(cluster_centers)

90

In [18]:
for c in cluster_centers:
    print(len(c))

6568
443
6008
5323
63
64
55
226
4488
38
54
229
248
83
35
99
38
49
33
46
24
111
40
30
113
138
48
202
24
103
31
30
185
33
17
24
32
25
110
113
54
170
88
178
19
145
126
32
21
13
0
22
104
143
228
44
24
78
111
0
74
79
25
133
154
50
41
142
6
18
8
26
122
40
6
73
28
20
16
108
42
156
58
103
95
19
44
80
103
0


In [20]:
import pickle
pickle.dump(c_model.return_clusters(), open( "Amazon_10_90_clusters_70.p", "wb" ) )

In [27]:
counts = np.zeros(90)
clusters = c_model.return_clusters() 
for c in clusters:
    counts[clusters[c]] += 1

for c in counts:
    print(floor(c))

30
13
48
124
3
3
6
8
320
2
3
8
11
5
5
9
3
3
5
3
4
8
4
3
6
6
4
8
4
8
4
2
7
5
3
2
4
2
4
3
7
5
8
13
2
7
5
3
3
2
0
2
3
8
13
4
2
5
8
0
6
8
4
9
5
6
4
10
1
3
1
3
7
3
1
4
3
3
2
6
6
12
6
8
4
2
4
8
10
0


In [32]:
clusters_inv = [[] for i in range(90)]
# clusters = c_model.return_clusters() 
for c in clusters:
    clusters_inv[clusters[c]] =clusters_inv[clusters[c]]  +  [c] 
#         print(c_models.users) 



In [33]:
clusters_inv[0]

[140,
 145,
 149,
 160,
 165,
 174,
 190,
 203,
 215,
 287,
 396,
 477,
 517,
 526,
 552,
 595,
 624,
 627,
 669,
 772,
 808,
 839,
 860,
 881,
 889,
 923,
 961,
 980,
 990,
 1000]

In [39]:
users = c_model.users
for c in users:
    if c.user_id in clusters_inv[0]:
        print(c.ratings)

{5267: 5.0, 6923: 4.0, 2820: 4.0, 8119: 5.0, 11999: 5.0, 10827: 5.0, 2032: 5.0, 13542: 4.0, 7599: 2.0}
{3000: 5.0, 4347: 5.0, 13959: 5.0, 8412: 4.0, 4957: 5.0, 11869: 4.0, 13167: 4.0, 834: 5.0, 12513: 4.0, 13932: 4.0, 14319: 5.0, 5953: 5.0, 15911: 5.0, 13044: 5.0, 6327: 3.0}
{7849: 4.0, 560: 5.0, 9282: 4.0, 6200: 5.0, 4054: 4.0, 13603: 5.0, 12596: 5.0, 13371: 5.0, 5908: 3.0, 15503: 4.0, 9210: 4.0, 5953: 5.0}
{4608: 5.0, 14371: 5.0, 4028: 5.0, 966: 5.0, 15443: 5.0, 10461: 5.0, 12876: 5.0, 982: 5.0, 1382: 5.0, 3009: 5.0, 6551: 5.0, 457: 5.0}
{4974: 5.0, 9016: 5.0, 11186: 5.0, 648: 5.0, 9746: 5.0, 5193: 5.0, 7173: 5.0, 10827: 5.0, 8813: 5.0, 3499: 5.0, 6449: 4.0, 3256: 5.0, 4151: 5.0}
{513: 5.0, 5616: 5.0, 2512: 5.0, 6985: 5.0, 13004: 5.0}
{14785: 5.0, 9993: 4.0, 6773: 5.0, 8770: 4.0, 6529: 4.0, 7678: 4.0}
{1080: 5.0, 7278: 5.0, 298: 5.0, 14149: 5.0, 4546: 5.0, 11734: 5.0, 7586: 5.0, 2831: 5.0, 1375: 5.0}
{3000: 4.0, 4150: 5.0, 5357: 5.0, 10930: 4.0, 4078: 2.0, 3910: 3.0, 4957: 5.0, 11869

## cluster with .05

In [40]:
from clustering import *

c_model = ClusteringModel(num_clusters=90, num_iterations=70)
c_model.fit(train)
